In [94]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [95]:
# Helper function to create dataframes for each trace output
def make_df(output_dir):
    stats = pd.DataFrame()
    for filename in os.listdir(output_dir):
        file_path = os.path.join(output_dir, filename)
        cache_info = []
        new_row = None
        with open(file_path, 'r') as file:
            lines = file.readlines()
            if lines:
                for line in lines:
                    cache_info.append(int(line.split(':')[1]))
                        
                info = filename.split('_')
                new_row = pd.Series({
                    'set_size': int(info[0]), 
                    'block_size': int(info[1]),
                    'byte_size': int(info[2]),
                    'write_policy': info[3], 
                    'write_through': info[4],
                    'cache_type': info[5],
                    'total loads': cache_info[0],
                    'total stores': cache_info[1],
                    'load hits': cache_info[2],
                    'load misses': cache_info[3],
                    'store hits': cache_info[4],
                    'store misses': cache_info[5],
                    'total cycles': cache_info[6]
                })
                stats = pd.concat([stats, new_row.to_frame().T])
        stats['load hit/miss ratio'] = stats['load hits'] / stats['load misses']
        stats['store hit/miss ratio'] = stats['store hits'] / stats['store misses']
        stats['cache size'] = stats['set_size'] * stats['block_size'] * stats['byte_size']
    return stats

In [96]:
gcc_dir = os.path.join(os.getcwd(), 'gcc_output')
gcc_data = make_df(gcc_dir)

swim_dir = os.path.join(os.getcwd(), 'swim_output')
swim_data = make_df(swim_dir)

In [97]:
print(f"Cache sizes:\n{swim_data['cache size'].unique()}")
print(swim_data.keys())

Cache sizes:
[64 8 1024 4096 512 32768 256 65536 16384 262144 32 2048 8192 128 131072
 16]
Index(['set_size', 'block_size', 'byte_size', 'write_policy', 'write_through',
       'cache_type', 'total loads', 'total stores', 'load hits', 'load misses',
       'store hits', 'store misses', 'total cycles', 'load hit/miss ratio',
       'store hit/miss ratio', 'cache size'],
      dtype='object')


In [98]:
def compute_stats_by_size(col_name, cache_size, data):
    sorted_df = data.sort_values(by=col_name, ascending=False)
    highest_load_ratio = sorted_df.iloc[0:5].drop(columns=['total loads', 'total stores', 'load hits', 'load misses',
       'store hits', 'store misses', 'total cycles', 'load hit/miss ratio',
       'store hit/miss ratio', 'cache size'])
    print(f"Top 5 rows for cache size {cache_size} with the highest load hit/miss ratio:\n{highest_load_ratio}")

In [99]:
test_cache_sizes = [1024, 4096, 16384, 262144]

In [100]:
# Compute stats for gcc 
for s in test_cache_sizes:
    compute_stats_by_size('load hit/miss ratio', s, gcc_data)


Top 5 rows for cache size 1024 with the highest load hit/miss ratio:
  set_size block_size byte_size    write_policy  write_through cache_type
0      256         64        16  write-allocate     write-back   fifo.txt
0      256         64        16  write-allocate     write-back    lru.txt
0      256         64        16  write-allocate  write-through   fifo.txt
0      256         64        16  write-allocate  write-through    lru.txt
0      128         64        16  write-allocate     write-back    lru.txt
Top 5 rows for cache size 4096 with the highest load hit/miss ratio:
  set_size block_size byte_size    write_policy  write_through cache_type
0      256         64        16  write-allocate     write-back   fifo.txt
0      256         64        16  write-allocate     write-back    lru.txt
0      256         64        16  write-allocate  write-through   fifo.txt
0      256         64        16  write-allocate  write-through    lru.txt
0      128         64        16  write-allocate 

In [101]:
for s in test_cache_sizes:
    compute_stats_by_size('store hit/miss ratio', s, gcc_data)

Top 5 rows for cache size 1024 with the highest load hit/miss ratio:
  set_size block_size byte_size    write_policy  write_through cache_type
0      256         64        16  write-allocate  write-through    lru.txt
0      256         64        16  write-allocate     write-back    lru.txt
0      256         64        16  write-allocate  write-through   fifo.txt
0      256         64        16  write-allocate     write-back   fifo.txt
0      128         64        16  write-allocate  write-through    lru.txt
Top 5 rows for cache size 4096 with the highest load hit/miss ratio:
  set_size block_size byte_size    write_policy  write_through cache_type
0      256         64        16  write-allocate  write-through    lru.txt
0      256         64        16  write-allocate     write-back    lru.txt
0      256         64        16  write-allocate  write-through   fifo.txt
0      256         64        16  write-allocate     write-back   fifo.txt
0      128         64        16  write-allocate 

In [102]:
for s in test_cache_sizes:
    compute_stats_by_size('load hit/miss ratio', s, swim_data)

Top 5 rows for cache size 1024 with the highest load hit/miss ratio:
  set_size block_size byte_size    write_policy  write_through cache_type
0      256         64        16  write-allocate  write-through   fifo.txt
0      256         64        16  write-allocate     write-back   fifo.txt
0      128         64        16  write-allocate     write-back    lru.txt
0      256         64        16  write-allocate  write-through    lru.txt
0      256         64        16  write-allocate     write-back    lru.txt
Top 5 rows for cache size 4096 with the highest load hit/miss ratio:
  set_size block_size byte_size    write_policy  write_through cache_type
0      256         64        16  write-allocate  write-through   fifo.txt
0      256         64        16  write-allocate     write-back   fifo.txt
0      128         64        16  write-allocate     write-back    lru.txt
0      256         64        16  write-allocate  write-through    lru.txt
0      256         64        16  write-allocate 

In [103]:
for s in test_cache_sizes:
    compute_stats_by_size('store hit/miss ratio', s, swim_data)

Top 5 rows for cache size 1024 with the highest load hit/miss ratio:
  set_size block_size byte_size    write_policy  write_through cache_type
0      128         64        16  write-allocate  write-through    lru.txt
0      256         64        16  write-allocate     write-back   fifo.txt
0      128         64        16  write-allocate     write-back    lru.txt
0      256         64        16  write-allocate  write-through   fifo.txt
0      256         64        16  write-allocate     write-back    lru.txt
Top 5 rows for cache size 4096 with the highest load hit/miss ratio:
  set_size block_size byte_size    write_policy  write_through cache_type
0      128         64        16  write-allocate  write-through    lru.txt
0      256         64        16  write-allocate     write-back   fifo.txt
0      128         64        16  write-allocate     write-back    lru.txt
0      256         64        16  write-allocate  write-through   fifo.txt
0      256         64        16  write-allocate 